<a href="https://colab.research.google.com/github/AudiXXXXX/AudiXXXXX/blob/main/flux.1-dev_jupyter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /content
!git clone -b totoro3 https://github.com/camenduru/ComfyUI /content/TotoroUI
%cd /content/TotoroUI

!pip install -q torchsde einops diffusers accelerate xformers==0.0.28.post2
!apt -y install -qq aria2

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/flux1-dev-fp8.safetensors -d /content/TotoroUI/models/unet -o flux1-dev-fp8.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/ae.sft -d /content/TotoroUI/models/vae -o ae.sft
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/clip_l.safetensors -d /content/TotoroUI/models/clip -o clip_l.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/t5xxl_fp8_e4m3fn.safetensors -d /content/TotoroUI/models/clip -o t5xxl_fp8_e4m3fn.safetensors

import random
import torch
import numpy as np
from PIL import Image
import nodes
from nodes import NODE_CLASS_MAPPINGS
from totoro_extras import nodes_custom_sampler
from totoro import model_management

DualCLIPLoader = NODE_CLASS_MAPPINGS["DualCLIPLoader"]()
UNETLoader = NODE_CLASS_MAPPINGS["UNETLoader"]()
RandomNoise = nodes_custom_sampler.NODE_CLASS_MAPPINGS["RandomNoise"]()
BasicGuider = nodes_custom_sampler.NODE_CLASS_MAPPINGS["BasicGuider"]()
KSamplerSelect = nodes_custom_sampler.NODE_CLASS_MAPPINGS["KSamplerSelect"]()
BasicScheduler = nodes_custom_sampler.NODE_CLASS_MAPPINGS["BasicScheduler"]()
SamplerCustomAdvanced = nodes_custom_sampler.NODE_CLASS_MAPPINGS["SamplerCustomAdvanced"]()
VAELoader = NODE_CLASS_MAPPINGS["VAELoader"]()
VAEDecode = NODE_CLASS_MAPPINGS["VAEDecode"]()
EmptyLatentImage = NODE_CLASS_MAPPINGS["EmptyLatentImage"]()

with torch.inference_mode():
    clip = DualCLIPLoader.load_clip("t5xxl_fp8_e4m3fn.safetensors", "clip_l.safetensors", "flux")[0]
    unet = UNETLoader.load_unet("flux1-dev-fp8.safetensors", "fp8_e4m3fn")[0]
    vae = VAELoader.load_vae("ae.sft")[0]

def closestNumber(n, m):
    q = int(n / m)
    n1 = m * q
    if (n * m) > 0:
        n2 = m * (q + 1)
    else:
        n2 = m * (q - 1)
    if abs(n - n1) < abs(n - n2):
        return n1
    return n2

/content
Cloning into '/content/TotoroUI'...
remote: Enumerating objects: 14652, done.
remote: Total 14652 (delta 0), reused 0 (delta 0), pack-reused 14652 (from 1)
Receiving objects: 100% (14652/14652), 23.92 MiB | 8.68 MiB/s, done.
Resolving deltas: 100% (9811/9811), done.
/content/TotoroUI
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s e

In [4]:
OUTPUT_FOLDER='/content/'
def generate_image(prompt, output_path):
  with torch.inference_mode():
    positive_prompt =prompt
    width = 1024
    height = 1024
    seed = 0
    steps = 20
    sampler_name = "euler"
    scheduler = "simple"

    if seed == 0:
        seed = random.randint(0, 18446744073709551615)
    print(seed)

    cond, pooled = clip.encode_from_tokens(clip.tokenize(positive_prompt), return_pooled=True)
    cond = [[cond, {"pooled_output": pooled}]]
    noise = RandomNoise.get_noise(seed)[0]
    guider = BasicGuider.get_guider(unet, cond)[0]
    sampler = KSamplerSelect.get_sampler(sampler_name)[0]
    sigmas = BasicScheduler.get_sigmas(unet, scheduler, steps, 1.0)[0]
    latent_image = EmptyLatentImage.generate(closestNumber(width, 16), closestNumber(height, 16))[0]
    sample, sample_denoised = SamplerCustomAdvanced.sample(noise, guider, sampler, sigmas, latent_image)
    model_management.soft_empty_cache()
    decoded = VAEDecode.decode(vae, sample)[0].detach()
    Image.fromarray(np.array(decoded*255, dtype=np.uint8)[0]).save(output_path)

  Image.fromarray(np.array(decoded*255, dtype=np.uint8)[0])

content ='''
"A manga scene showing the boy joyfully seeing flags of his clan on two ships. His face drops to despair as a monstrous roar echoes from the ocean, followed by a gigantic ancient beast, the Eight-Clawed Tyrant Dragon, bursting through the waves. The ships are violently crushed in an explosion of water and debris."//The Ancient Beast’s Attack
"A dynamic manga action scene where the Eight-Clawed Tyrant Dragon, with a sharp horn and glowing silvery scales, crashes into two ships, slicing them in half. Water splashes dramatically as the monster’s giant body looms above the wreckage. Survivors are scattered in the ocean, some trying to escape."//Survivors Reaching the Beach
"A tense manga panel showing the few survivors, including the boy, struggling to swim to the beach, drenched and exhausted. The ocean is chaotic, with huge waves and debris, while the Eight-Clawed Tyrant Dragon remains in the distance, its presence still felt.//Girls First Trap: Silver CentipedeA manga scene where the girl and another person cautiously approach a rocky hill. A giant silver centipede, with gleaming scales and sharp fangs, bursts from the ground and attacks, tearing apart the two people in front with lightning speed. Sparks fly as weapons fail to pierce its body.//The Skeletons Arrival A dark, eerie manga panel showing the boy waking up in a dense, gloomy swamp. Three skeletons with glowing eye sockets hang upside down like bats from a tree, their bones creaking as they land silently on the ground, observing the boy with curiosity."//Naming the Skeletons"A manga panel where the boy, sitting cross-legged on an ancient tree, names the three skeletons. The skeletons bow in acknowledgment as their glowing eyes flicker with a mysterious energy."//The Skeletons’ Curiosity"A manga scene depicting the skeletons following the boy out of the swamp into a sunlit forest. The three skeletons are shown walking cautiously in the shadows, avoiding sunlight, while the boy leads them toward a giant panther lurking nearby."//Battle with the Giant Panther"An intense manga action scene showing the boy and his three skeleton allies fighting a massive five-meter-long panther. The skeletons use their razor-sharp claws to pierce the panther’s skull, killing it instantly. The boy stands back, observing with a satisfied look."//Finding the Crystal"A manga panel showing the boy extracting a sparkling yellow crystal from the dead panthers corpse. The three skeletons eagerly stare at the crystal, their eye sockets glowing with hunger as the boy holds it up triumphantly."//Girl and Boy Closing In"A dramatic manga scene showing the girl and the boy leading their group through the dense jungle, their faces filled with malice. In the background, the boy sits quietly in the swamp, his eyes narrowing as he senses their approach. The three skeletons are hidden nearby, ready for action.//A dramatic manga panel showing the three skeletons, Qinguang Wang, Yanluo Wang, and Lunhui Wang, lying silently in the swamp, surrounded by dozens of scattered skeletons. The boy, Xiao Chen, watches in disbelief as the skeletons spring into action, taking advantage of the moment to attack their enemies with vicious bone claws.//A beautiful, ethereal scene of Zhao Lin Er, with skin as white as snow, walking gracefully through the swamp, her beauty like that of a fallen goddess. Behind her, the brown-haired man, Gu Luo, leads a group of experts, their faces filled with apprehension as they step into the eerie, foggy swamp//A dark and foreboding manga spread showing the swamp filled with ghastly skeletons, their glowing eyes scattered through the fog. The aura of death is thick, and the experts are visibly nervous as they move deeper into the land of death.//A close-up manga panel of Gu Luo's face, his lips curling into a cold smile. He stands confidently as he surveys the swamp, believing he’s finally cornered his enemy. His eyes narrow with cold calculation as he prepares for the inevitable confrontation.//A shocking manga moment where a white skeleton leaps from the ground, its bone claws aiming for Gu Luo’s chest. The look of surprise and panic on his face is stark against the eerie, silent swamp. The bone claws pierce the air with incredible speed.//A high-intensity manga action scene showing Gu Luo fiercely trying to block the attack with his fists. Skeletons are wrapping around his legs, holding him in place while another bone claw pierces through his back. His face contorts in pain as blood spills into the swamp.//A manga panel showing Zhao Lin Er's face pale with fear as she watches the violent attack on Gu Luo. Her graceful features are marred with shock as she draws her sword, determined to fight back against the undead creatures.//A swift manga action scene where the three skeletons, Qinguang Wang, Yanluo Wang, and Lunhui Wang, retreat in perfect coordination after their vicious attack on Gu Luo. They stab their bone claws deep into his body once more, leaving bloody holes as they fade into the shadows.//A powerful manga panel showing Xiao Chen leaping from a giant tree and stomping the ground, causing the two attackers to collapse with shattered sternums. His movement is swift, leaving afterimages as he enters the fray, preparing to deliver a deadly blow//A dynamic manga spread showing the three skeletons, Qinguang Wang, Yanluo Wang, and Lunhui Wang, moving through the chaotic battlefield like streaks of white light. Their bone claws pierce through the chests of their enemies as they move with incredible speed, causing mayhem in the swamp'''

# Split prompts by //
prompts = content.split("//")

# Process each prompt
j=0
for i, prompt in enumerate(prompts):
    j=j+1
    prompt = prompt.strip()
    if prompt:  # Skip empty lines
        output_path = f"{OUTPUT_FOLDER}image_{j + 1}.png"
        generate_image(prompt, output_path)


3420286669301055763


  0%|          | 0/20 [00:00<?, ?it/s]

10669575894914487963


  0%|          | 0/20 [00:00<?, ?it/s]

3302670071208496308


...

  0%|          | 0/20 [00:00<?, ?it/s]

4475806941760758473


  0%|          | 0/20 [00:00<?, ?it/s]

18363496371019097553


  0%|          | 0/20 [00:00<?, ?it/s]

In [6]:
import cv2
import os
import numpy as np

def create_video_from_images(image_folder, output_file, display_time=10):
    # Get all .png and .jpg files from the folder and sort them by their modification time (oldest to newest)
    images = [f for f in os.listdir(image_folder) if f.endswith('.png') or f.endswith('.jpg')]

    # Sort images by their modification time
    images.sort(key=lambda x: os.path.getmtime(os.path.join(image_folder, x)))  # Sort by modification time

    # Check if any images are found
    if not images:
        print("No PNG or JPG images found in the folder.")
        return

    # Read the first image to get the dimensions for the video
    first_image = cv2.imread(os.path.join(image_folder, images[0]))
    height, width, layers = first_image.shape

    # Set the video writer with the same dimensions as the images
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Using 'mp4v' codec for .mp4 format
    video_writer = cv2.VideoWriter(output_file, fourcc, 1.0, (width, height))

    # Iterate through each image and write it to the video
    for image_name in images:
        image_path = os.path.join(image_folder, image_name)
        image = cv2.imread(image_path)

        # Write the image to the video for the specified display time
        for _ in range(int(display_time * 30)):  # Assuming 30 fps, multiply by 30 to get frame count
            video_writer.write(image)

    # Release the video writer object
    video_writer.release()
    print(f"Video saved as {output_file}")

# Example usage:
folder_path = '/content'  # Replace with the path to your folder
output_video_path = '/content/output_video.mp4'  # The path where the video will be saved

create_video_from_images(folder_path, output_video_path, display_time=1)  # Each image will display for 10 seconds


Video saved as /content/output_video.mp4
